In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 16

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda() 
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = 2*torch.rand(Nb, dimension) - 1.0
    xa1[:, 0] = -torch.ones(Nb)
    
    xa2 = 2*torch.rand(Nb, dimension) - 1.0
    xa2[:, 0:16] = xa1[:, 0:16]
    xa2[:, 0] = torch.ones(Nb)
    
    xb1 = 2*torch.rand(Nb, dimension) - 1.0
    xb1[:, 1] = -torch.ones(Nb)
    
    xb2 = 2*torch.rand(Nb, dimension) - 1.0
    xb2[:, 0:16] = xb1[:, 0:16] 
    xb2[:, 1] = torch.ones(Nb)
    
    xc1 = 2*torch.rand(Nb, dimension) - 1.0
    xc1[:, 2] = -torch.ones(Nb)
    
    xc2 = 2*torch.rand(Nb, dimension) - 1.0
    xc2[:, 0:16] = xc1[:, 0:16]
    xc2[:, 2] = torch.ones(Nb)
    
    xd1 = 2*torch.rand(Nb, dimension) - 1.0
    xd1[:, 3] = -torch.ones(Nb)
    
    xd2 = 2*torch.rand(Nb, dimension) - 1.0
    xd2[:, 0:16] = xd1[:, 0:16]
    xd2[:, 3] = torch.ones(Nb)
    
    xe1 = (2*torch.rand(Nb, dimension) - 1.0)
    xe1[:, 4] = -torch.ones(Nb)
    
    xe2 = (2*torch.rand(Nb, dimension) - 1.0)
    xe2[:, 0:16] = xe1[:, 0:16]
    xe2[:, 4] = torch.ones(Nb)
    
    xf1 = 2*torch.rand(Nb, dimension) - 1.0
    xf1[:, 5] = -torch.ones(Nb)
    
    xf2 = 2*torch.rand(Nb, dimension) - 1.0
    xf2[:, 0:16] = xf1[:, 0:16]
    xf2[:, 5] = torch.ones(Nb)
    
    xg1 = 2*torch.rand(Nb, dimension) - 1.0
    xg1[:, 6] = -torch.ones(Nb)
    
    xg2 = 2*torch.rand(Nb, dimension) - 1.0
    xg2[:, 0:16] = xg1[:, 0:16] 
    xg2[:, 6] = torch.ones(Nb)
    
    xh1 = 2*torch.rand(Nb, dimension) - 1.0
    xh1[:, 7] = -torch.ones(Nb)
    
    xh2 = 2*torch.rand(Nb, dimension) - 1.0
    xh2[:, 0:16] = xh1[:, 0:16]
    xh2[:, 7] = torch.ones(Nb)
    
    xi1 = 2*torch.rand(Nb, dimension).cuda() - 1.0  
    xi1[:, 8] = -torch.ones(Nb)
    
    xi2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xi2[:, 0:16] = xi1[:, 0:16]
    xi2[:, 8] = torch.ones(Nb)
    
    xj1 = 2*torch.rand(Nb, dimension).cuda() - 1.0 
    xj1[:, 9] = -torch.ones(Nb)
    
    xj2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xj2[:, 0:16] = xj1[:, 0:16]
    xj2[:, 9] = torch.ones(Nb)
    
    xk1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xk1[:, 10] = -torch.ones(Nb)
    
    xk2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xk2[:, 0:16] = xk1[:, 0:16]
    xk2[:, 10] = torch.ones(Nb)
    
    xl1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xl1[:, 11] = -torch.ones(Nb)
    
    xl2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xl2[:, 0:16] = xl1[:, 0:16]
    xl2[:, 11] = torch.ones(Nb)
    
    xm1 = 2*torch.rand(Nb, dimension).cuda() - 1.0 
    xm1[:, 12] = -torch.ones(Nb)
    
    xm2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xm2[:, 0:16] = xm1[:, 0:16]
    xm2[:, 12] = torch.ones(Nb)
    
    xn1 = 2*torch.rand(Nb, dimension).cuda() - 1.0 
    xn1[:, 13] = -torch.ones(Nb)
    
    xn2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xn2[:, 0:16] = xn1[:, 0:16]
    xn2[:, 13] = torch.ones(Nb)
    
    xo1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xo1[:, 14] = -torch.ones(Nb)
    
    xo2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xo2[:, 0:16] = xo1[:, 0:16]
    xo2[:, 14] = torch.ones(Nb)
    
    xp1 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xp1[:, 15] = -torch.ones(Nb)
    
    xp2 = 2*torch.rand(Nb, dimension).cuda() - 1.0
    xp2[:, 0:16] = xp1[:, 0:16]
    xp2[:, 15] = torch.ones(Nb)
    
    xa1 = xa1.cuda()
    xa2 = xa2.cuda()
    xb1 = xb1.cuda()
    xb2 = xb2.cuda()
    xc1 = xc1.cuda()
    xc2 = xc2.cuda()
    xd1 = xd1.cuda()
    xd2 = xd2.cuda()
    xe1 = xe1.cuda()
    xe2 = xe2.cuda()
    xf1 = xf1.cuda()
    xf2 = xf2.cuda()
    xg1 = xg1.cuda()
    xg2 = xg2.cuda()
    xh1 = xh1.cuda()
    xh2 = xh2.cuda()
    xi1 = xi1.cuda()
    xi2 = xi2.cuda()
    xj1 = xj1.cuda()
    xj2 = xj2.cuda()
    xk1 = xk1.cuda()
    xk2 = xk2.cuda()
    xl1 = xl1.cuda()
    xl2 = xl2.cuda()
    xm1 = xm1.cuda()
    xm2 = xm2.cuda()
    xn1 = xn1.cuda()
    xn2 = xn2.cuda()
    xo1 = xo1.cuda()
    xo2 = xo2.cuda()
    xp1 = xp1.cuda()
    xp2 = xp2.cuda()
    
    pxa = ((model(xa1) - model(xa2))**2).cuda() 
    pxb = ((model(xb1) - model(xb2))**2).cuda()  
    pxc = ((model(xc1) - model(xc2))**2).cuda()  
    pxd = ((model(xd1) - model(xd2))**2).cuda()  
    pxe = ((model(xe1) - model(xe2))**2).cuda() 
    pxf = ((model(xf1) - model(xf2))**2).cuda()  
    pxg = ((model(xg1) - model(xg2))**2).cuda()  
    pxh = ((model(xh1) - model(xh2))**2).cuda()
    pxi = ((model(xi1) - model(xi2))**2).cuda() 
    pxj = ((model(xj1) - model(xj2))**2).cuda()  
    pxk = ((model(xk1) - model(xk2))**2).cuda()  
    pxl = ((model(xl1) - model(xl2))**2).cuda()  
    pxm = ((model(xm1) - model(xm2))**2).cuda() 
    pxn = ((model(xn1) - model(xn2))**2).cuda()  
    pxo = ((model(xo1) - model(xo2))**2).cuda()  
    pxp = ((model(xp1) - model(xp2))**2).cuda()
    
    part_2 = torch.sum((pxa + pxb + pxc + pxd + pxe + pxf + pxg + pxh + pxi + pxj + pxk + pxl + pxm + pxn + pxo + pxp)[:,0])/2/xa1.size()[0]/16
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    dx9 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx9[:, 8] = torch.ones(xb1.size()[0])
    dx10 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx10[:, 9] = torch.ones(xb1.size()[0])
    dx11 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx11[:, 10] = torch.ones(xb1.size()[0])
    dx12 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx12[:, 11] = torch.ones(xb1.size()[0])
    dx13 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx13[:, 12] = torch.ones(xb1.size()[0])
    dx14 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx14[:, 13] = torch.ones(xb1.size()[0])
    dx15 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx15[:, 14] = torch.ones(xb1.size()[0])
    dx16 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx16[:, 15] = torch.ones(xb1.size()[0])
    
    dxa = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 - (model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2)**2).cuda()  
    dxb = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 - (model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2)**2).cuda()  
    dxc = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 - (model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2)**2).cuda()  
    dxd = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 - (model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2)**2).cuda()  
    dxe = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 - (model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2)**2).cuda()  
    dxf = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 - (model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2)**2).cuda()  
    dxg = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 - (model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2)**2).cuda()  
    dxh = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 - (model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2)**2).cuda()  
    dxi = (((model(xi1+step_size*dx9) - model(xi1-step_size*dx9))/step_size/2 - (model(xi2+step_size*dx9) - model(xi2-step_size*dx9))/step_size/2)**2).cuda()  
    dxj = (((model(xj1+step_size*dx10) - model(xj1-step_size*dx10))/step_size/2 - (model(xj2+step_size*dx10) - model(xj2-step_size*dx10))/step_size/2)**2).cuda()  
    dxk = (((model(xk1+step_size*dx11) - model(xk1-step_size*dx11))/step_size/2 - (model(xk2+step_size*dx11) - model(xk2-step_size*dx11))/step_size/2)**2).cuda()  
    dxl = (((model(xl1+step_size*dx12) - model(xl1-step_size*dx12))/step_size/2 - (model(xl2+step_size*dx12) - model(xl2-step_size*dx12))/step_size/2)**2).cuda()  
    dxm = (((model(xm1+step_size*dx13) - model(xm1-step_size*dx13))/step_size/2 - (model(xm2+step_size*dx13) - model(xm2-step_size*dx13))/step_size/2)**2).cuda()  
    dxn = (((model(xn1+step_size*dx14) - model(xn1-step_size*dx14))/step_size/2 - (model(xn2+step_size*dx14) - model(xn2-step_size*dx14))/step_size/2)**2).cuda()  
    dxo = (((model(xo1+step_size*dx15) - model(xo1-step_size*dx15))/step_size/2 - (model(xo2+step_size*dx15) - model(xo2-step_size*dx15))/step_size/2)**2).cuda()  
    dxp = (((model(xp1+step_size*dx16) - model(xp1-step_size*dx16))/step_size/2 - (model(xp2+step_size*dx16) - model(xp2-step_size*dx16))/step_size/2)**2).cuda()  
    
    part_3 = torch.sum((dxa + dxb + dxc + dxd + dxe + dxf + dxg + dxh + dxi + dxj + dxk + dxl + dxm + dxn + dxo + dxp)[:,0])/2/xb1.size()[0]/16
    
    lambda1 = 10
    lambda2 = 5
    return part_1 + lambda1 * part_2 + lambda2 * part_3

In [10]:
Data_size = 2000
def Gendata():
    x = 2*torch.rand(Data_size, dimension) - 1.0
    return x.cuda()  

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 32)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0")
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=16, out_features=32, bias=True)
  (linear1): Linear(in_features=32, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=32, bias=True)
  (linear3): Linear(in_features=32, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=32, bias=True)
  (linear5): Linear(in_features=32, out_features=32, bias=True)
  (linear6): Linear(in_features=32, out_features=32, bias=True)
  (linear_out): Linear(in_features=32, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 100000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_16D_Periodic.npy", error_save)
np.save("DGM_relative_error_16D_Periodic.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(34417783.6852, device='cuda:0')
current relative error is:  tensor(67.6362, device='cuda:0')
current epoch is:  50
current loss is:  tensor(22548837.7691, device='cuda:0')
current relative error is:  tensor(41.9580, device='cuda:0')
current epoch is:  100
current loss is:  tensor(14646793.3289, device='cuda:0')
current relative error is:  tensor(27.1036, device='cuda:0')
current epoch is:  150
current loss is:  tensor(10391625.6863, device='cuda:0')
current relative error is:  tensor(18.9952, device='cuda:0')
current epoch is:  200
current loss is:  tensor(8051681.9269, device='cuda:0')
current relative error is:  tensor(17.2708, device='cuda:0')
current epoch is:  250
current loss is:  tensor(5830701.8454, device='cuda:0')
current relative error is:  tensor(13.3410, device='cuda:0')
current epoch is:  300
current loss is:  tensor(3693464.6496, device='cuda:0')
current relative error is:  tensor(11.7743, device='cuda:0')
current epoch is:  

current epoch is:  3300
current loss is:  tensor(39901.4058, device='cuda:0')
current relative error is:  tensor(1.7805, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(31721.3097, device='cuda:0')
current relative error is:  tensor(1.7461, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(33092.8042, device='cuda:0')
current relative error is:  tensor(1.7937, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(30997.8910, device='cuda:0')
current relative error is:  tensor(1.7909, device='cuda:0')
current epoch is:  3500
current loss is:  tensor(42816.0005, device='cuda:0')
current relative error is:  tensor(1.8031, device='cuda:0')
current epoch is:  3550
current loss is:  tensor(46555.8114, device='cuda:0')
current relative error is:  tensor(1.7594, device='cuda:0')
current epoch is:  3600
current loss is:  tensor(30358.8292, device='cuda:0')
current relative error is:  tensor(1.6968, device='cuda:0')
current epoch is:  3650
current lo

current epoch is:  6300
current loss is:  tensor(10050.8972, device='cuda:0')
current relative error is:  tensor(1.4265, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(9885.2298, device='cuda:0')
current relative error is:  tensor(1.3230, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(9804.1778, device='cuda:0')
current relative error is:  tensor(1.3407, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(11098.6693, device='cuda:0')
current relative error is:  tensor(1.3498, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(9438.0520, device='cuda:0')
current relative error is:  tensor(1.3116, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(9634.7474, device='cuda:0')
current relative error is:  tensor(1.3669, device='cuda:0')
current epoch is:  6600
current loss is:  tensor(8740.4369, device='cuda:0')
current relative error is:  tensor(1.3214, device='cuda:0')
current epoch is:  6650
current loss is

current epoch is:  9300
current loss is:  tensor(3732.1104, device='cuda:0')
current relative error is:  tensor(1.0926, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(3744.9936, device='cuda:0')
current relative error is:  tensor(1.1321, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(3789.4513, device='cuda:0')
current relative error is:  tensor(1.1085, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(3836.0212, device='cuda:0')
current relative error is:  tensor(1.1436, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(3573.7117, device='cuda:0')
current relative error is:  tensor(1.0886, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(3578.4559, device='cuda:0')
current relative error is:  tensor(1.1015, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(3745.4168, device='cuda:0')
current relative error is:  tensor(1.0688, device='cuda:0')
current epoch is:  9650
current loss is: 

current epoch is:  12300
current loss is:  tensor(2852.8335, device='cuda:0')
current relative error is:  tensor(1.0273, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(2905.2223, device='cuda:0')
current relative error is:  tensor(1.0539, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(2960.0388, device='cuda:0')
current relative error is:  tensor(1.0445, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(2880.0120, device='cuda:0')
current relative error is:  tensor(0.9663, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(2814.1939, device='cuda:0')
current relative error is:  tensor(1.1349, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(2800.5864, device='cuda:0')
current relative error is:  tensor(1.0163, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(2722.0432, device='cuda:0')
current relative error is:  tensor(1.0050, device='cuda:0')
current epoch is:  12650
current l

...

current epoch is:  85200
current loss is:  tensor(2.7817, device='cuda:0')
current relative error is:  tensor(0.0414, device='cuda:0')
current epoch is:  85250
current loss is:  tensor(2.1744, device='cuda:0')
current relative error is:  tensor(0.0449, device='cuda:0')
current epoch is:  85300
current loss is:  tensor(2.8646, device='cuda:0')
current relative error is:  tensor(0.0689, device='cuda:0')
current epoch is:  85350
current loss is:  tensor(2.8127, device='cuda:0')
current relative error is:  tensor(0.0480, device='cuda:0')
current epoch is:  85400
current loss is:  tensor(2.4209, device='cuda:0')
current relative error is:  tensor(0.0454, device='cuda:0')
current epoch is:  85450
current loss is:  tensor(2.5857, device='cuda:0')
current relative error is:  tensor(0.0573, device='cuda:0')
current epoch is:  85500
current loss is:  tensor(2.4765, device='cuda:0')
current relative error is:  tensor(0.0424, device='cuda:0')
current epoch is:  85550
current loss is:  tensor(2.554

current epoch is:  88250
current loss is:  tensor(2.3061, device='cuda:0')
current relative error is:  tensor(0.0513, device='cuda:0')
current epoch is:  88300
current loss is:  tensor(2.8654, device='cuda:0')
current relative error is:  tensor(0.0579, device='cuda:0')
current epoch is:  88350
current loss is:  tensor(3.1457, device='cuda:0')
current relative error is:  tensor(0.0588, device='cuda:0')
current epoch is:  88400
current loss is:  tensor(3.1669, device='cuda:0')
current relative error is:  tensor(0.0537, device='cuda:0')
current epoch is:  88450
current loss is:  tensor(2.5916, device='cuda:0')
current relative error is:  tensor(0.0528, device='cuda:0')
current epoch is:  88500
current loss is:  tensor(2.6591, device='cuda:0')
current relative error is:  tensor(0.0506, device='cuda:0')
current epoch is:  88550
current loss is:  tensor(3.0295, device='cuda:0')
current relative error is:  tensor(0.0573, device='cuda:0')
current epoch is:  88600
current loss is:  tensor(2.181

current epoch is:  91300
current loss is:  tensor(3.0881, device='cuda:0')
current relative error is:  tensor(0.0623, device='cuda:0')
current epoch is:  91350
current loss is:  tensor(2.4104, device='cuda:0')
current relative error is:  tensor(0.0453, device='cuda:0')
current epoch is:  91400
current loss is:  tensor(3.0660, device='cuda:0')
current relative error is:  tensor(0.0554, device='cuda:0')
current epoch is:  91450
current loss is:  tensor(2.7761, device='cuda:0')
current relative error is:  tensor(0.0523, device='cuda:0')
current epoch is:  91500
current loss is:  tensor(2.9970, device='cuda:0')
current relative error is:  tensor(0.0511, device='cuda:0')
current epoch is:  91550
current loss is:  tensor(2.9808, device='cuda:0')
current relative error is:  tensor(0.0580, device='cuda:0')
current epoch is:  91600
current loss is:  tensor(2.8207, device='cuda:0')
current relative error is:  tensor(0.0501, device='cuda:0')
current epoch is:  91650
current loss is:  tensor(2.307

current epoch is:  94350
current loss is:  tensor(2.7730, device='cuda:0')
current relative error is:  tensor(0.0549, device='cuda:0')
current epoch is:  94400
current loss is:  tensor(2.2783, device='cuda:0')
current relative error is:  tensor(0.0454, device='cuda:0')
current epoch is:  94450
current loss is:  tensor(2.3690, device='cuda:0')
current relative error is:  tensor(0.0555, device='cuda:0')
current epoch is:  94500
current loss is:  tensor(2.7259, device='cuda:0')
current relative error is:  tensor(0.0544, device='cuda:0')
current epoch is:  94550
current loss is:  tensor(2.7539, device='cuda:0')
current relative error is:  tensor(0.0650, device='cuda:0')
current epoch is:  94600
current loss is:  tensor(2.5676, device='cuda:0')
current relative error is:  tensor(0.0586, device='cuda:0')
current epoch is:  94650
current loss is:  tensor(2.4875, device='cuda:0')
current relative error is:  tensor(0.0540, device='cuda:0')
current epoch is:  94700
current loss is:  tensor(2.399

current epoch is:  97400
current loss is:  tensor(2.8918, device='cuda:0')
current relative error is:  tensor(0.0580, device='cuda:0')
current epoch is:  97450
current loss is:  tensor(4.0589, device='cuda:0')
current relative error is:  tensor(0.0519, device='cuda:0')
current epoch is:  97500
current loss is:  tensor(1.9916, device='cuda:0')
current relative error is:  tensor(0.0438, device='cuda:0')
current epoch is:  97550
current loss is:  tensor(2.1220, device='cuda:0')
current relative error is:  tensor(0.0403, device='cuda:0')
current epoch is:  97600
current loss is:  tensor(2.7562, device='cuda:0')
current relative error is:  tensor(0.0584, device='cuda:0')
current epoch is:  97650
current loss is:  tensor(2.6141, device='cuda:0')
current relative error is:  tensor(0.0497, device='cuda:0')
current epoch is:  97700
current loss is:  tensor(2.2239, device='cuda:0')
current relative error is:  tensor(0.0471, device='cuda:0')
current epoch is:  97750
current loss is:  tensor(2.228